<a href="https://colab.research.google.com/github/zamudation/Capstone_Files/blob/main/Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this section I'm just readind the file using the pandas liabrary

In [ ]:
import pandas as pd

In [ ]:
!wget https://raw.githubusercontent.com/zamudation/Capstone_Files/main/movies.csv

--2022-12-07 22:06:51--  https://raw.githubusercontent.com/zamudation/Capstone_Files/main/movies.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3038099 (2.9M) [text/plain]
Saving to: ‘movies.csv’

movies.csv          100%[===================>]   2.90M  --.-KB/s    in 0.05s   

2022-12-07 22:06:51 (59.7 MB/s) - ‘movies.csv’ saved [3038099/3038099]



In [ ]:
movies = pd.read_csv("movies.csv")

In [ ]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


Here I'm cleaning the data on the title colum and creating a new one. This will later be use to change it to a vector and use machine learning on it.

In [ ]:
import re

def cleanTitle(title):
  return re.sub("[^a-zA-Z0-9 ]", "", title)

In [ ]:
movies["cleanTitle"] = movies["title"].apply(cleanTitle)

In [ ]:
movies

,movieId,title,genres,cleanTitle
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
62418,209157,We (2018),Drama,We 2018
62419,209159,Window of the Soul (2001),Documentary,Window of the Soul 2001
62420,209163,Bad Poems (2018),Comedy|Drama,Bad Poems 2018
62421,209169,A Girl Thing (2001),(no genres listed),A Girl Thing 2001


Her I'm transforming the titles into vectors to be analized and use.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(movies["cleanTitle"])

This search function is used to find the title that are consider similar to the one that was tyeped. This will later be use to be analized and use on the movie recommendation system

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
  title = cleanTitle(title)
  queryVector = vectorizer.transform([title])
  similarity = cosine_similarity(queryVector, tfidf).flatten()
  indices = np.argpartition(similarity, -5)[-5:]
  results = movies.iloc[indices][::-1]
  return results

This is just a test section to make sure the search function works correctly.

In [ ]:
import ipywidgets as widgets
from IPython.display import display

movieInput = widgets.Text(
    value = "The Hulk",
    description  = "Movie Title",
    disabled = False
)

movieList = widgets.Output()

def onType(data):
  with movieList:
    movieList.clear_output()
    title = data["new"]
    if len(title) > 5:
      display(search(title))

movieInput.observe(onType, names="value")

display(movieInput, movieList)

Text(value='The Hulk', description='Movie Title')

Output()

Now we will read the information on the ratings file which contain the ratings users have given to different movies on the plataform. This will be neccesary and important to de machine learning part of the project to work correctly.

In [ ]:
!wget https://media.githubusercontent.com/media/zamudation/Capstone_Files/main/ratings.csv

--2022-12-07 22:06:57--  https://media.githubusercontent.com/media/zamudation/Capstone_Files/main/ratings.csv
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 678260987 (647M) [text/plain]
Saving to: ‘ratings.csv’

ratings.csv         100%[===================>] 646.84M   215MB/s    in 3.0s    

2022-12-07 22:07:21 (215 MB/s) - ‘ratings.csv’ saved [678260987/678260987]



In [ ]:
ratings = pd.read_csv("ratings.csv")

In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


In [ ]:
ratings.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

This couple blocks of code are the developement of the function that will recommend the movies. The reason of this section it was to make sure that the project was working as wanted and to develope the project littel by little.

In [ ]:
movieId = 345

In [ ]:
similarUsers = ratings[(ratings["movieId"] == movieId) & (ratings["rating"] >= 5)]["userId"].unique()

In [ ]:
similarUsers

array([   123,    189,    190, ..., 162324, 162473, 162483])

In [ ]:
similarUserRecs = ratings[(ratings["userId"].isin(similarUsers)) & (ratings["rating"] > 4)]["movieId"]

In [ ]:
similarUserRecs

15849        345
15893       1446
15897       1527
15912       1912
15933       2542
            ... 
24985275    1230
24985277    1235
24985278    1244
24985279    1356
24985280    1449
Name: movieId, Length: 123456, dtype: int64

In [ ]:
similarUserRecs = similarUserRecs.value_counts() / len(similarUsers)

similarUserRecs = similarUserRecs[similarUserRecs > .1]

In [ ]:
similarUserRecs

345     1.000000
296     0.408284
593     0.402959
318     0.385799
527     0.383432
          ...   
1784    0.101775
720     0.101775
1271    0.101183
1358    0.101183
1252    0.100592
Name: movieId, Length: 130, dtype: float64

In [ ]:
allUsers = ratings[(ratings["movieId"].isin(similarUserRecs.index)) & (ratings["rating"] > 4)]

In [ ]:
allUsersRecs = allUsers["movieId"].value_counts() / len(allUsers["userId"].unique())

In [ ]:
allUsersRecs

318     0.345654
296     0.287530
2571    0.246482
356     0.237626
593     0.228175
          ...   
1188    0.014220
1296    0.013859
232     0.012996
342     0.012849
272     0.010173
Name: movieId, Length: 130, dtype: float64

In [ ]:
recPercentages = pd.concat([similarUserRecs, allUsersRecs], axis=1)
recPercentages.columns = ["Similar", "All"]

In [ ]:
recPercentages

,Similar,All
1,0.194675,0.125980
17,0.213609,0.046038
21,0.117751,0.024420
25,0.130178,0.033824
32,0.190533,0.101299
...,...,...
3996,0.116568,0.064732
4226,0.108876,0.133277
4973,0.113609,0.113532
4993,0.127811,0.189395


In [ ]:
recPercentages["score"] = recPercentages["Similar"] / recPercentages["All"]

In [ ]:
recPercentages = recPercentages.sort_values("score", ascending=False)

In [ ]:
recPercentages

,Similar,All,score
345,1.000000,0.014789,67.620081
342,0.203550,0.012849,15.841956
272,0.122485,0.010173,12.039789
1188,0.165089,0.014220,11.609638
232,0.139645,0.012996,10.745260
...,...,...,...
4226,0.108876,0.133277,0.816912
2028,0.105917,0.132856,0.797234
2571,0.170414,0.246482,0.691387
4993,0.127811,0.189395,0.674838


In [ ]:
recPercentages.head(10).merge(movies, left_index=True, right_on="movieId")

,Similar,All,score,movieId,title,genres,cleanTitle
340,1.000000,0.014789,67.620081,345,"Adventures of Priscilla, Queen of the Desert, ...",Comedy|Drama,Adventures of Priscilla Queen of the Desert Th...
337,0.203550,0.012849,15.841956,342,Muriel's Wedding (1994),Comedy,Muriels Wedding 1994
269,0.122485,0.010173,12.039789,272,"Madness of King George, The (1994)",Comedy|Drama,Madness of King George The 1994
1159,0.165089,0.014220,11.609638,1188,Strictly Ballroom (1992),Comedy|Romance,Strictly Ballroom 1992
229,0.139645,0.012996,10.745260,232,Eat Drink Man Woman (Yin shi nan nu) (1994),Comedy|Drama|Romance,Eat Drink Man Woman Yin shi nan nu 1994
262,0.230178,0.023865,9.645006,265,Like Water for Chocolate (Como agua para choco...,Drama|Fantasy|Romance,Like Water for Chocolate Como agua para chocol...
1067,0.155030,0.016949,9.146868,1094,"Crying Game, The (1992)",Drama|Romance|Thriller,Crying Game The 1992
510,0.156805,0.019758,7.936209,515,"Remains of the Day, The (1993)",Drama|Romance,Remains of the Day The 1993
1579,0.173373,0.021865,7.929219,1641,"Full Monty, The (1997)",Comedy|Drama,Full Monty The 1997
504,0.214201,0.028580,7.494685,509,"Piano, The (1993)",Drama|Romance,Piano The 1993


After every part of the code was tested. The next findSimilarmMovies function contains all the the code sections above but now as a function. This is because everything has been tested and now the main function can be added knowing there won't be issue and it will work as intended.

In [ ]:
def findSimilarMovies(movieId):
  similarUsers = ratings[(ratings["movieId"] == movieId) & (ratings["rating"] >= 5)]["userId"].unique()
  similarUserRecs = ratings[(ratings["userId"].isin(similarUsers)) & (ratings["rating"] > 4)]["movieId"]

  similarUserRecs = similarUserRecs.value_counts() / len(similarUsers)
  similarUserRecs = similarUserRecs[similarUserRecs > .1]

  allUsers = ratings[(ratings["movieId"].isin(similarUserRecs.index)) & (ratings["rating"] > 4)]
  allUsersRecs = allUsers["movieId"].value_counts() / len(allUsers["userId"].unique())

  recPercentages = pd.concat([similarUserRecs, allUsersRecs], axis=1)
  recPercentages.columns = ["Similar", "All"]

  recPercentages["score"] = recPercentages["Similar"] / recPercentages["All"]

  recPercentages = recPercentages.sort_values("score", ascending=False)
  return recPercentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]

These are the widgets to interact with the program. Bwllow the code the interactive section for the program is provided.

In [ ]:
movieInputName = widgets.Text(
    value = "Please type a movie you like",
    description = "Movie Title",
    disabled = False
    
)

recommendationList = widgets.Output()

def onType(data):
  with recommendationList:
    recommendationList.clear_output()
    title = data["new"]
    if len(title) >= 4:
      results = search(title)
      movieId = results.iloc[0]["movieId"]
      display(findSimilarMovies(movieId))

movieInputName.observe(onType, names="value")
display(movieInputName, recommendationList)

Text(value='Please type a movie you like', description='Movie Title')

Output()